# 0) Download Required Libraries

In [11]:
!pip install transformers
import transformers
import numpy as np
import random
import re
from math import *


# 1) Initialize parameters

In [12]:
DEBUG=False


SECRET_MESSAGE="بمطعم العم"
SECRET_MESSAGE="مرحبا كيفك"
SECRET_MESSAGE="الموعد عند الساعه السابعه"


from enum import Enum
class SourceCoding(Enum):
    HUFFMAN = 1
    BINARY  = 3

class TextGenerationLang(Enum):
    Arabic  = 1

class ModelName(Enum):
    Roberta = 7

class StegoPriority(Enum):
    Capacity=1
    Efficiency=2
    Imperceptibility=3

top_k=32
sourceCodingType=SourceCoding.HUFFMAN
TextGenerationLang=TextGenerationLang.Arabic
choosenModel=ModelName.Roberta
choosenStegoPriority=StegoPriority.Capacity

# Discard the first ? words because they were added to control the context and do not have any data hidden inside them.
# This is a constant number that should match between sender and receiver before lunch. (Same as length of senderGeneratedText variable in Sender)
numberOfContextWordAdded=4

# 3) Models

## You Model

In [13]:
if choosenModel==ModelName.Roberta:
  from transformers import pipeline,TFAutoModelForMaskedLM,TFBertForMaskedLM
  from transformers import AutoTokenizer

  # For example, this model is not good; you should use your trained model instead.
  checkpoint="xlm-roberta-base"

  model_name=checkpoint

  tokenizer = AutoTokenizer.from_pretrained(checkpoint)

  def fillArabicMaskWithRoberta(text,top_k=30):
    text=text+" <mask>"
    model = pipeline(task ="fill-mask",model=model_name,tokenizer=tokenizer,top_k =top_k,device=0)
    return model(text)




# 4) Enconding

## 1) Huffman Coding

In [14]:
# A Huffman Tree Node
class Node:
    def __init__(self, prob, symbol, left=None, right=None):
        # probability of symbol
        self.prob = prob

        # symbol
        self.symbol = symbol

        # left node
        self.left = left

        # right node
        self.right = right

        # tree direction (0/1)
        self.code = ''

""" A helper function to print the codes of symbols by traveling Huffman Tree"""
codes = dict()

def Calculate_Codes(node, val=''):
    # huffman code for current node
    newVal = val + str(node.code)

    if(node.left):
        Calculate_Codes(node.left, newVal)
    if(node.right):
        Calculate_Codes(node.right, newVal)

    if(not node.left and not node.right):
        codes[node.symbol] = newVal

    return codes

""" A helper function to calculate the probabilities of symbols in given data"""
def Calculate_Probability(data):
    symbols = dict()
    for element in data:
        if symbols.get(element) == None:
            symbols[element] = 1
        else:
            symbols[element] += 1
    return symbols

""" A helper function to obtain the encoded output"""
def Output_Encoded(data, coding):
    encoding_output = []
    for c in data:
      #  print(coding[c], end = '')
        encoding_output.append(coding[c])

    string = ''.join([str(item) for item in encoding_output])
    return string

""" A helper function to calculate the space difference between compressed and non compressed data"""
def Total_Gain(data, coding):
    before_compression = len(data) * 8 # total bit space to stor the data before compression
    after_compression = 0
    symbols = coding.keys()
    for symbol in symbols:
        count = data.count(symbol)
        after_compression += count * len(coding[symbol]) #calculate how many bit is required for that symbol in total
    if DEBUG:
      print("Space usage before compression (in bits):", before_compression)
      print("Space usage after compression (in bits):",  after_compression)

def Huffman_Encoding(data):
    symbol_with_probs = Calculate_Probability(data)
    symbols = symbol_with_probs.keys()
    probabilities = symbol_with_probs.values()
    if DEBUG:
      print("symbols: ", symbols)
      print("probabilities: ", probabilities)


    nodes = []
    # converting symbols and probabilities into huffman tree nodes
    for symbol in symbols:
        nodes.append(Node(symbol_with_probs.get(symbol), symbol))

    while len(nodes) > 1:
        # sort all the nodes in ascending order based on their probability
        nodes = sorted(nodes, key=lambda x: x.prob)
        # for node in nodes:
        #      print(node.symbol, node.prob)

        # pick 2 smallest nodes
        right = nodes[0]
        left = nodes[1]

        left.code = 0
        right.code = 1

        # combine the 2 smallest nodes to create new node
        newNode = Node(left.prob+right.prob, left.symbol+right.symbol, left, right)

        nodes.remove(left)
        nodes.remove(right)
        nodes.append(newNode)

    huffman_encoding = Calculate_Codes(nodes[0])
    if DEBUG:
      print("symbols with codes", huffman_encoding)

    Total_Gain(data, huffman_encoding)
    encoded_output = Output_Encoded(data,huffman_encoding)
    return encoded_output, nodes[0],huffman_encoding


def Huffman_Decoding(encoded_data, huffman_tree,wordSplit=False):
    tree_head = huffman_tree
    decoded_output = []
    for x in encoded_data:
        if x == '1':
            huffman_tree = huffman_tree.right
        elif x == '0':
            huffman_tree = huffman_tree.left
        try:
            if huffman_tree.left.symbol == None and huffman_tree.right.symbol == None:
                pass
        except AttributeError:
            decoded_output.append(huffman_tree.symbol)
            huffman_tree = tree_head

    delimiter= ' ' if wordSplit else ''
    string = delimiter.join([str(item) for item in decoded_output])
    return string





In [15]:

def encodeByHuffman(data,wordSplit=False):
  data=data.strip()
  if wordSplit:
    data=data.split(' ')

  global codes
  codes=dict()
  encoding, tree,mapping = Huffman_Encoding(data)
  return (encoding,tree,mapping)

def decodeByHuffman(encoding,tree,wordSplit=False):
  return Huffman_Decoding(encoding,tree,wordSplit)



## 2) Binary Tree Coding

In [16]:
def buildBinaryTreeCoding(words,currCode=''):

  l=len(words)

  if log2(l)!= log2(pow(2,int(log2(l)))):
    print('The Length Is Not Power Of 2')
    return None

  if l==1:
    return [currCode]
    # return [(words[0],currCode)]

  mid=int(l/2)
  a=buildBinaryTreeCoding(words[:mid],currCode+'0')
  b=buildBinaryTreeCoding(words[mid:],currCode+'1')

  return a+b

def binaryTreeEncoding(words):
  l=len(words)
  leafNb=int(pow(2,int(log2(l))))
  restToCeilLog=int(pow(2,ceil(log2(l))))-leafNb
  words=words+['NO_TOKEN_'+str(i) for i in range(1,restToCeilLog)]
  if DEBUG==True:
    print(len(words))
    print(words)
  wordsCode=buildBinaryTreeCoding(words)
  mapWordToCode= {k:v  for k,v in zip(words,wordsCode) }
  encoding=''.join(list(map(lambda x:mapWordToCode[x],words)))
  return encoding,mapWordToCode



In [17]:
def encodeByBinaryTree(data,wordSplit=False):
  data=data.strip()
  if wordSplit:
    data=data.split(' ')
  else:
    data=list(set(data))

  encoding,mapping = binaryTreeEncoding(data)
  return (encoding,mapping)


# OO ) Main

### Common Functions

In [18]:
def fillMask(generatedText,top_k=top_k):
  top_k=top_k*2
  masks=[]
  if choosenModel==ModelName.Roberta:
    masks=fillArabicMaskWithRoberta(generatedText,top_k=top_k)
  return masks

In [19]:
def generateNextMaskedWord(generatedText,wordIdx,sourceCodingType,top_kth):
  tok_k=top_kth

  if TextGenerationLang==TextGenerationLang.Arabic:
    substitution=r'[^\u0620-\u064a ]*'
  elif TextGenerationLang==TextGenerationLang.English:
    substitution=r'[^a-zA-Z ]*'

  masks=fillMask(generatedText,top_k=top_k)
  masks=list(map(lambda ele: re.sub(substitution, '', ele["sequence"].split(" ")[-1]),masks))

  if DEBUG==True:
    print("Masks: ",masks)

  lastWord=''
  if DEBUG==True:
    print("last element of genereted text : " )
    print(generatedText.split(" "))

  if len(generatedText.split(" "))>0:
    if DEBUG==True:
      print("last element of genereted text : "+ generatedText.split(" ")[-1])
    lastWord=generatedText.split(" ")[-1]
  masks=list(filter(lambda ele: (ele!='' and len(ele)!=1 and ele!=lastWord) ,masks))
  masks=masks[0:max(len(masks),int(top_k/2))]

  encoding=mapWordToCode=None
  if sourceCodingType==SourceCoding.HUFFMAN:
    masks=' '.join(masks)
    encoding,_,mapWordToCode=encodeByHuffman(masks,True)
  elif sourceCodingType==SourceCoding.BINARY:
    l=len(masks)
    leafNb=int(pow(2,int(log2(l))))
    if DEBUG==True:
      print('leafNb: ',leafNb)
    masks=masks[:leafNb]
    # if DEBUG==True:
    print("edit len mask : ",len(masks))
    masks=' '.join(masks)
    encoding,mapWordToCode=encodeByBinaryTree(masks,True)

  mapCodeToWord=dict(map(reversed, mapWordToCode.items()))
  return (encoding,mapWordToCode,mapCodeToWord)

## Sender

In [31]:
SECRET_MESSAGE = re.sub('[^\u0620-\u064a ]+', '', SECRET_MESSAGE)
mapLetterToCode = {'ل': '0000', 'ص': '000100', 'ج': '0001010', 'ز': '0001011', 'أ': '000110', 'ف': '000111', 'ك': '001000', 'إ': '00100100', 'ظ': '0010010100', 'آ': '0010010101', 'ء': '001001011', 'ش': '0010011', 'ب': '00101', 'و': '00110', 'ح': '0011100', 'خ': '0011101', 'د': '001111',
                  'م': '01000', 'ع': '01001', 'ط': '0101000', 'ذ': '0101001', 'ى': '0101010', 'ض': '0101011', 'ر': '01011', 'ي': '0110', 'ة': '011100', 'س': '011101', 'ت': '01111', 'ق': '100000', 'ث': '10000100', 'غ': '10000101', 'ئ': '1000011', 'ه': '10001', 'ن': '1001', 'ا': '101', ' ': '11'}
encodingSecretMessage = ''.join(list(map(lambda x: mapLetterToCode[x], SECRET_MESSAGE)))

END_TOKEN_CODE = "0"*8
encodingSecretMessage = encodingSecretMessage+END_TOKEN_CODE
if DEBUG==True:
  print(encodingSecretMessage)

In [32]:
# To control the context that the model should commit to, you may add words from a prefix of a phrase by selecting a sentence at random from your list.
# However, keep in mind that the sentence's length should equal numberOfContextWordAdded variable
senderGeneratedText='أعلن الريس على عن'

if DEBUG==True:
  print(len(senderGeneratedText.split(' ')))
  print(numberOfContextWordAdded)

if(len(senderGeneratedText.split(' '))!=numberOfContextWordAdded):
  raise RuntimeError("numberOfContextWordAdded should equal the length of senderGeneratedText")

wordIdx=0
while len(encodingSecretMessage):
  encoding,mapWordToCode,mapCodeToWord=generateNextMaskedWord(senderGeneratedText,wordIdx,sourceCodingType,top_k)
  wordIdx=wordIdx+1
  if DEBUG==True:
    print(mapCodeToWord.keys())
    print(mapCodeToWord)


  encodingSecretMessageLength=len(encodingSecretMessage)
  chooseWord=False
  for i in range(0,encodingSecretMessageLength):
    senderCurrEncodingMatchTest=encodingSecretMessage[0:encodingSecretMessageLength-i]


    if mapCodeToWord.get(senderCurrEncodingMatchTest)==None:
      continue

    senderChoosenWord=mapCodeToWord.get(senderCurrEncodingMatchTest)
    if DEBUG==True:
      print(senderCurrEncodingMatchTest +" ----> "+senderChoosenWord)

    if len(senderGeneratedText)!=0:
      senderGeneratedText=senderGeneratedText+" "
    senderGeneratedText=senderGeneratedText+senderChoosenWord
    encodingSecretMessage=encodingSecretMessage[encodingSecretMessageLength-i:]
    if DEBUG==True:
      print("rest encoding secrete message : "+encodingSecretMessage)
      print("length rest encoding secrete message : "+str(len(encodingSecretMessage)))
    chooseWord=True
    break

  if DEBUG==True:
    print("out for loop with choose word value : "+str(chooseWord))

  if chooseWord==False:
    keys=mapCodeToWord.keys()
    keys=list(keys)
    keys.sort(reverse=True)
    keys=' '.join(keys)
    x = re.search("\s"+encodingSecretMessage+".*\s", keys)
    keys=keys[x.start():]
    senderCurrEncodingMatchTest=keys.split(" ")[1]
    if DEBUG==True:
      print("***************************")
      print(senderCurrEncodingMatchTest)
    senderChoosenWord=mapCodeToWord.get(senderCurrEncodingMatchTest)
    if len(senderGeneratedText)!=0:
     senderGeneratedText=senderGeneratedText+' '
    senderGeneratedText=senderGeneratedText+senderChoosenWord
    encodingSecretMessage=""

    if DEBUG==True:
      print("***************************")

if DEBUG==True:
  print("length of secret message enconding : "+str(len(encodingSecretMessage)))
  print(encodingSecretMessage)




print("senderGeneratedText : "+senderGeneratedText)


Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertFor

senderGeneratedText : أعلن الريس على عن حياته قريبا السبت الجمعة الماضي إلى حين تاريخ النهاية مع العدالة المتحدة الجمعة قبل الغد وبعد الأربعاء هذا هذان هذانث هذانثة هذانثةه هو
